
# 404—GEN | SUBNET 17

This notebook demonstrates how the 404-GEN subnet works — a decentralized network where miners generate 3D PLY files with Gaussian splats based on either text or image prompts.
Miners pull generation tasks directly from validators, create 3D outputs, and submit their results for validation. Validators then provide feedback, apply cooldowns or penalties, and use duel-based comparisons (via LLMs) to update miner ratings.
Through this demo, you’ll see how tasks are distributed, validated, and rewarded — illustrating the full miner lifecycle within the subnet.

## Miner Workflow Overview

Each miner operates independently, continuously requesting generation tasks from validators, producing PLY outputs, and submitting results for validation. The validator provides feedback that determines when the miner can submit the next task.

The pseudocode below shows the complete miner loop, including task retrieval, result submission, and cooldown handling.

In [ ]:
import asyncio
import base64
import time

import bittensor as bt
import pyspz
from open3d import shell

from common.protocol import PullTask, SubmitResults, ProtocolTask

MINER_LICENSE_CONSENT_DECLARATION = (
    "I, as a miner on SN17, have obtained all licenses, rights and consents required to use, reproduce, "
    "modify, display, distribute and make available my submitted results to this subnet and its end users"
)


async def pull_task(validator_uid: int, wallet: bt.wallet, metagraph: bt.metagraph) -> PullTask:
    async with bt.dendrite(wallet=wallet) as dendrite:
        synapse = PullTask()
        return await dendrite.call(
            target_axon=metagraph.axons[validator_uid], synapse=synapse, deserialize=False, timeout=12.0
        )


async def submit_results(
    validator_uid: int, wallet: bt.wallet, metagraph: bt.metagraph, task: ProtocolTask, compressed_results: str
) -> SubmitResults:
    async with bt.dendrite(wallet=wallet) as dendrite:
        submit_time = time.time_ns()
        message = (
            f"{MINER_LICENSE_CONSENT_DECLARATION}"
            f"{submit_time}{task.prompt}{metagraph.hotkeys[validator_uid]}{wallet.hotkey.ss58_address}"
        )
        signature = base64.b64encode(dendrite.keypair.sign(message)).decode("utf-8")
        synapse = SubmitResults(
            task_id=task.id, results=compressed_results, submit_time=submit_time, signature=signature
        )
        return await dendrite.call(
            target_axon=metagraph.axons[validator_uid], synapse=synapse, deserialize=False, timeout=30.0
        )


async def work_with_validator(validator_uid: int):
    wallet = bt.wallet()
    subtensor = bt.subtensor(network="finney")
    metagraph = bt.metagraph(netuid=17, network=subtensor.network, sync=True)

    while True:
        synapse = await pull_task(validator_uid=validator_uid, wallet=wallet, metagraph=metagraph)
        ply = await generate_ply(task_type=synapse.task.type, prompt=synapse.task.prompt)

        if ply is None:  # failed to generate or low quality
            feedback = await submit_results(
                validator_uid=validator_uid,
                wallet=wallet,
                metagraph=metagraph,
                task=synapse.task,
                compressed_results="",
            )
        else:
            compressed = pyspz.compress(ply)
            encoded = base64.b64encode(compressed).decode("utf-8")
            feedback = await submit_results(
                validator_uid=validator_uid,
                wallet=wallet,
                metagraph=metagraph,
                task=synapse.task,
                compressed_results=encoded,
            )

        await asyncio.sleep(feedback.cooldown_until)


This loop demonstrates the miner’s end-to-end interaction with the subnet — pulling tasks, generating 3D outputs, submitting results, and respecting cooldown feedback from the validator.

## Protocol and Challenge Types

The protocol defines how miners and validators exchange tasks and results.
Each miner pulls a task, generates a 3D PLY, submits results, and receives structured feedback.
Tasks can be either text-based or image-based, represented by two models combined into a single union type.

In [ ]:
import uuid
from typing import Literal

from pydantic import BaseModel, Field


class TextTask(BaseModel):
    type: Literal["text"] = "text"
    id: str = Field(default_factory=lambda: str(uuid.uuid4()), description="Unique task identifier")
    prompt: str = Field(default="", description="Prompt text for 3D model generation")


class ImageTask(BaseModel):
    type: Literal["image"] = "image"
    id: str = Field(default_factory=lambda: str(uuid.uuid4()), description="Unique task identifier")
    prompt: str = Field(default="", description="Base64-encoded image data for 3D model generation")


ProtocolTask = TextTask | ImageTask
ProtocolTaskType = Literal["text", "image"]

Miners receive a PullTask response that defines task parameters and timing controls.
Miners submit results through the SubmitResults model, which also carries validator feedback.

In [ ]:
from typing import Any
import base64
import time


class Feedback(BaseModel):
    """Feedback model for miner task validation and performance tracking."""

    validation_failed: bool = Field(default=False, description="True if the validator rejected this task")
    task_fidelity_score: float = Field(default=0.0, description="Fidelity score (0.0-1.0)")
    average_fidelity_score: float = Field(default=0.0, description="Exponential moving average fidelity score")
    generations_within_the_window: int = Field(default=0, description="Accepted generations in the last 4 hours")
    current_duel_rating: float = Field(default=0.0, description="Current Glicko2 duel rating")
    current_miner_reward: float = Field(default=0.0, description="Latest calculated reward value")


class PullTask(bt.Synapse):
    """Miner requesting a new task from the validator."""

    task: ProtocolTask | None = Field(default=None, description="Task assigned by validator")
    validation_threshold: float = Field(
        default=0.6,
        description="Minimum fidelity score required for acceptance. "
        "Below-threshold results are rejected and penalized."
    )
    throttle_period: int = Field(
        default=0,
        description="Minimum expected completion time in seconds. "
        "Used to compute cooldown adjustments. Detailed later in *Key Concepts*."
    )
    cooldown_until: int = Field(
        default=0, description="Unix timestamp when the miner can request the next task"
    )
    cooldown_violations: int = Field(
        default=0,
        description="Number of times the miner violated cooldown rules. Explained later in *Key Concepts*."
    )


class SubmitResults(bt.Synapse):
    """Miner submitting generation results."""

    task: ProtocolTask | None = Field(default=None, description="[Deprecated] Original task reference")
    task_id: str = Field(default="", description="Identifier of the task being submitted")
    results: str = Field(description="Generated PLY data, encoded as a string")
    data_format: str = Field(default="ply", description="Reserved for future use")
    data_ver: int = Field(default=0, description="Reserved for future use")
    compression: int = Field(
        default=2,
        description="Compression method identifier (spz compression required: https://github.com/404-Repo/spz)",
    )
    submit_time: int = Field(description="Submission timestamp in nanoseconds")
    signature: str = Field(
        description="Miner signature: "
        "b64encode(sign(f'{MINER_LICENSE_CONSENT_DECLARATION}{submit_time}{prompt}{validator.hotkey}{miner.hotkey}'))"
    )
    feedback: Feedback | None = Field(default=None, description="Feedback provided by the validator")
    cooldown_until: int = Field(
        default=0, description="UTC time when miner can pull the next task from this validator"
    )

This schema defines the complete communication contract between miners and validators.

## Core Mechanics

This section describes how the subnet regulates miner behavior and performance through timing, validation, and feedback mechanisms. These parameters determine how efficiently a miner can operate and how rewards are assigned.

### Cooldown
After submitting results, each miner enters a cooldown period defined by `cooldown_until`.
During this time, the miner cannot request new tasks from the same validator.
Cooldowns help balance the load and give validators the time to complete validation and duels.
### Cooldown Penalty
If a miner’s result fails validation, an additional penalty is applied to extend the cooldown.
Submitting **empty results** avoids this penalty but yields no reward.

### Cooldown Violation
If a miner ignores the cooldown limit and attempts to pull tasks prematurely, the validator applies `cooldown_violations`.
After exceeding a configured threshold (typically 100), the miner’s cooldown is temporarily increased as a deterrent.

### Throttle Period
Each task defines a `throttle_period`, typically 30 seconds, representing the expected completion window.

Cooldown is reduced by the miner’s actual task duration, up to the throttle limit.
Finishing too early does not shorten cooldown beyond this limit.

Example with 60s base cooldown and 20s throttle:
* 5s completion → 55s cooldown
* 15s completion → 45s cooldown
* 20s or more → 40s cooldown

### Task Repull
Miners may re-pull the same task multiple times.
This supports miner restarts, migration between hosts, or recovery from network errors.
A miner can safely re-pull a task and resubmit identical results to ensure the submission is accepted.

### Duels and Rating
Validators periodically compare results from two miners for the same prompt using an LLM.
The comparison outcome determines the winner of a **duel**, and both miners’ ratings are updated using the **Glicko2** system.

Higher rating reflects consistent quality relative to peers.

### Rewards
Each miner’s reward is proportional to both performance and consistency:
```
miner_reward = duel_rating × accepted_results_in_last_4h
```
The rating captures quality, while the result count captures throughput.

### Optimal Miner Behavior
A high-performing miner should:
* Win duels consistently (high fidelity and quality).
* Complete each generation within 30s.
* Request new task immediately after cooldown expires.
* With a 300s cooldown, this allows ~48 tasks in 4 hours;
with 120s cooldown, up to 120 tasks in 4 hours..


## Traffic Generation
The subnet relies on a continuous stream of prompts to evaluate miner performance under realistic conditions.
Prompts are sourced from two complementary sources:
* Synthetic Prompts: Autonomously generated and distributed by validators.
* Organic Prompts: Real user submissions received through the public gateway.
This design enables both reproducible benchmarking and exposure to naturally occurring prompt patterns.

## Synthetic Prompts
Synthetic prompts are produced through a distributed prompt generation pipeline.
Validators are expected to operate their own generators and collectors or connect to existing trusted endpoints.
This flexibility ensures that the subnet remains operational and diverse, regardless of who contributes prompt traffic at any given time.

### Prompt Generators
Two independent generation mechanisms are available, each balancing control and diversity.
#### Prompt Generator 1
Uses Three Rotating LLMs:
```
Qwen/Qwen2.5-7B-Instruct-1M
microsoft/phi-4
THUDM/glm-4-9b-chat-1m
```
Each model follows a predefined prompt-instruction, ensuring generated prompts are relevant for 3D asset creation.
Objects are selected from a curated list of categories (currently 30), derived from industry analysis and marketplace trends.
These categories can be expanded or refined to align with evolving 3D generation needs.
#### Prompt Generator 1
Focuses on generating natural, human-like prompts.
It randomly samples initial letters for both object category and object name, prompting the LLM to:
1. Select a matching category and object.
2. Write a concise, descriptive one-sentence prompt.
This slower but more expressive method introduces diversity and realism into the prompt dataset.

### Prompt Collector
The Prompt Collector aggregates outputs from one or more prompt generators and serves large prompt batches to validators.
Validators typically refresh batches on an hourly basis, though the interval is configurable.

## Organic Requests
Organic traffic represents real user submissions received through the public gateway infrastructure. These requests originate from external clients and are processed with priority over synthetic prompts, ensuring the best user experience.
### Gateway Architecture
The gateway system is built on a distributed consensus architecture using Raft (OpenRaft) to achieve coordination across nodes. The transport protocol leverages QUIC for fast, low-latency communication while encrypting all data, and the API layer operates over HTTP/3 for modern web standards compliance.

The solution employs a fully non-blocking concurrency model with lock-free algorithms throughout, designed to support high throughput capable of handling over 100,000 user requests per second.

All nodes in the cluster maintain a synchronized global state through log replication. This state includes each node's domain, IP address, available task count, and timing metadata, enabling intelligent load balancing and routing decisions. When validators request tasks, they receive both the task data and the current cluster state, allowing them to optimize subsequent requests based on factors like geographic proximity, queue depth, and latency.

Validators initially source tasks from the geographically closest gateway node to minimize latency, but implement dynamic load balancing to pull tasks from distant nodes when necessary to maintain overall system performance. Organic traffic maintains priority status and is directed to miners first, preserving the core workflow.

### Client Integration
External clients interact with the gateway through a straightforward HTTP/3 API. All requests require an API key passed via the `x-api-key` header. Gateway nodes are distributed across regions:
* gateway-eu.404.xyz (Europe)
* gateway-us-east.404.xyz (US East)
* gateway-us-west.404.xyz (US West)

### Add Task
Submit a text prompt for 3D generation:
```shell
curl --http3 -X POST "https://gateway-eu.404.xyz:4443/add_task" \
     -H "content-type: application/json" \
     -H "x-api-key: 123e4567-e89b-12d3-a456-426614174001" \
     -d '{"prompt": "mechanic robot"}'
```
Alternative form data syntax for text prompts:
```shell
curl --http3 -X POST "https://gateway-eu.404.xyz:4443/add_task" \
     -F "prompt=a robot" \
     -H "x-api-key: 123e4567-e89b-12d3-a456-426614174001"
```
Submit an image for 3D generation:
```shell
curl --http3 -X POST "https://gateway-eu.404.xyz:4443/add_task" \
     -F "image=@image.jpg" \
     -H "x-api-key: 123e4567-e89b-12d3-a456-426614174001"
```
Check Task Status:
```shell
curl --http3 "https://gateway-eu.404.xyz:4443/get_status?id=550e8400-e29b-41d4-a716-446655440000" \
     -H "x-api-key: 123e4567-e89b-12d3-a456-426614174001"
```
Retrieve Result:
```shell
curl --http3 "https://gateway-eu.404.xyz:4443/get_result?id=550e8400-e29b-41d4-a716-446655440000" \
     -H "x-api-key: 123e4567-e89b-12d3-a456-426614174001" \
     -o result.spz
```
Results are returned in compressed SPZ format. Decompress using the pyspz library: https://github.com/404-Repo/spz.


## Validator Algorithm

The validator assesses miner submissions using a multi-metric pipeline that evaluates prompt alignment and visual quality. The process varies by task type (`text-to-gs` vs `image-to-gs`) but follows a consistent scoring framework.

### Validation Pipeline

1. Decompress and load the submitted PLY file
2. Render multiple views of the 3D Gaussian splat
3. Apply task-specific alignment metrics (OpenCLIP for `text-to-gs`, LLM2CLIP for `image-to-gs`)
4. Compute quality and similarity metrics (DINOv2-based iqa, lpips,ssim)
5. Aggregate scores into a final fidelity value

### Metrics

- **alignment**: OpenCLIP (`text-to-gs`), LLM2CLIP (`image-to-gs`)
- **iqa**: DINOv2-based quality classifier (75% weight)
- **lpips**: Perceptual consistency across views
- **ssim**: Structural consistency across views

Submissions with alignment < 0.3 are rejected.